In [1]:
import pandas as pd
from darts import TimeSeries

In [2]:
import pandas as pd
from pandas import Series
from pandas.tseries.offsets import DateOffset
import numpy as np
import scipy
from scipy import stats
import scipy.stats as scs
import itertools
from scipy.special import boxcox, inv_boxcox

In [3]:
import statsmodels
import statsmodels.api as sm
import statsmodels.stats as sms
from statsmodels.stats.stattools import jarque_bera
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.api as smt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics import tsaplots
import pmdarima as pm

In [4]:
from datetime import datetime, timedelta, date

In [5]:
import prophet
from prophet import Prophet

/Users/IRIS/.pyenv/versions/3.10.6/envs/assist_trader/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import sys
import warnings
import matplotlib.cbook
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=matplotlib.cbook.mplDeprecation)


import pandas as pd
from pandas import Series
from pandas.tseries.offsets import DateOffset
import numpy as np
import scipy
from scipy import stats
import scipy.stats as scs
import itertools
from scipy.special import boxcox, inv_boxcox


import matplotlib.pyplot as plt
import matplotlib as m
import seaborn as sns

# Data Source
import yfinance as yf

# Stats models
import statsmodels
import statsmodels.api as sm
import statsmodels.stats as sms
from statsmodels.stats.stattools import jarque_bera
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.api as smt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics import tsaplots
import pmdarima as pm

# Date tools
from datetime import datetime, timedelta, date

# Prophet model
import prophet
from prophet import Prophet

# Imports for model metrics and performance
import math
from math import sqrt 
import sklearn
from sklearn.metrics import mean_squared_error
import plotly as py
import plotly.express as px
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.preprocessing import MinMaxScaler

In [7]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [8]:
import re

In [9]:
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf

from sklearn.metrics import mean_squared_error

In [10]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
plt.rcParams["figure.figsize"] = (14,6)
plt.style.use('fivethirtyeight')
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
m.rcParams['axes.labelsize'] = 14
m.rcParams['xtick.labelsize'] = 12
m.rcParams['ytick.labelsize'] = 12
m.rcParams['text.color'] = 'k'

In [12]:
data_path = ("../raw_data/clean_raw_data.csv")
data = pd.read_csv(data_path)
data.head()

,timestamp,open,high,low,close,volume,CMF,FII,MFI,VWA,AOI,KAMA,PPO,PPO_signal,PVO,PVO_signal,ROC,rsi,stoch_RSI,StochOscillator,StochOscillator_signal,TSI,UltimateOscillator,WilliamsRIndicator,ATR,BOL_lband,BOL_hband,BOL_avg,KCW,KCP,KCM,DCL,DCH,DCM,ULI,ADX,ADX_neg,ADX_pos,ARI_down,ARI,ARI_up,CCI,DPO,EMA_14,EMA_20,EMA_50,EMA_100,EMA_200,KST,KST_diff,KST_sig,MACD_line,MACD_signal,MAI,PSAR,SMA_14,SMA_20,SMA_50,SMA_100,SMA_200,STC,TRIX,vortex_ind_diff,vortex_neg,vortex_pos,wma_9,wma_20,wma_50,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-17 04:00:00,10216.35,10238.95,10194.39,10209.45,872.764,-0.120924,-913.266701,28.652105,10185.127432,-5.729912,10184.207475,-0.155468,-0.243303,1.037447,0.552894,0.598202,46.271721,0.676076,51.773363,58.093400,-10.639675,50.178602,-48.226637,52.300392,10073.766700,10358.878300,10216.3225,1.029615,0.418821,10217.990500,10080.7,10329.38,10205.040,1.452368,36.183688,30.858004,21.418009,60.0,-52.0,8.0,-3.877403,-79.6825,10216.809022,10223.590398,10253.805665,10258.150614,10223.457350,-10.009420,0.867360,-10.876780,-15.906877,-24.904204,26.531786,10100.005486,10181.362143,10216.3225,10269.6334,10292.0137,10247.86650,97.040222,-0.038235,-0.167639,1.075610,0.907971,10203.142000,10203.709333,10246.597545,10275.790895,3005.7,2.25,1.7,1,0.45992,0.46011,-1,48.386232,-1
1,2019-09-17 05:00:00,10208.81,10232.07,10204.53,10223.97,819.923,-0.114688,917.954536,36.501645,10187.270366,-12.684824,10186.551361,-0.137502,-0.222143,0.532956,0.548907,0.861528,48.644318,0.743204,57.612192,54.702161,-9.336736,54.098656,-42.387808,50.531792,10075.379977,10349.352023,10212.3660,1.031969,0.594886,10213.968500,10080.7,10329.38,10222.540,1.372308,34.888995,29.656738,20.584231,56.0,-52.0,4.0,6.904724,-92.2960,10217.763819,10223.626551,10252.635639,10257.473770,10223.462451,-9.424722,1.525039,-10.949762,-14.067871,-22.736938,26.587424,10104.592976,10186.853571,10212.3660,10267.6942,10290.3989,10248.98635,98.520111,-0.035367,0.026085,1.064804,1.090889,10208.823048,10204.437667,10244.806824,10274.443495,3005.7,2.25,1.7,-1,0.40934,0.45992,-1,42.873022,-1
2,2019-09-17 06:00:00,10224.11,10230.44,10187.88,10224.09,922.804,-0.112046,802.637671,36.306243,10191.939033,-26.571000,10188.754395,-0.121765,-0.202067,1.083131,0.655751,1.027859,48.664494,0.743775,57.660447,55.682001,-8.306903,51.810427,-42.339553,49.962379,10077.635517,10339.275483,10208.4555,1.049208,0.632549,10209.891000,10080.7,10329.38,10222.540,1.288889,33.967875,30.232613,19.331768,52.0,-44.0,8.0,5.018498,-64.1855,10218.607310,10223.670689,10251.516202,10256.812705,10223.468695,-8.210570,2.548815,-10.759384,-12.457165,-20.680983,26.638149,10109.088717,10192.235714,10208.4555,10265.5230,10288.4885,10248.40620,99.260055,-0.032511,0.148340,0.933562,1.081902,10213.787905,10205.554238,10243.096855,10273.130448,3005.7,2.25,1.7,-1,0.31251,0.40934,-1,42.291553,-1
3,2019-09-17 07:00:00,10225.02,10230.79,10192.69,10193.85,905.598,-0.124361,-3224.208213,35.631136,10195.919801,-33.398088,10188.963245,-0.131656,-0.187985,1.339276,0.792456,0.488749,43.975832,0.611118,45.500241,53.590960,-8.853256,45.944458,-54.499759,49.115066,10078.361917,10328.766083,10203.5640,1.059908,0.394859,10205.222667,10080.7,10329.38,10224.335,1.199061,33.103085,28.557447,18.311510,48.0,-44.0,4.0,0.707552,-58.8340,10215.306335,10220.830623,10249.254783,10255.565919,10223.173982,-7.215594,3.074639,-10.290233,-13.465561,-19.237899,26.537221,10113.494542,10196.322143,10203.5640,10262.9314,10286.5536,10247.61835,99.630028,-0.030300,0.189927,0.914300,1.104227,10214.003143,10204.163238,10240.286149,10271.256418,3005.7,2.25,1.7,-1,0.14629,0.31251,-1,41.595938,-1
4,2019-09-17 08:00:00,10193.21,10215.07,10165.00,10195.82,1089.829,-0.114965,-2456.898021,34.774165,10199.535893,-36.033412,10189.290069,-0.136376,-0.177663,3.169482,1.267861,0.503611,44.351977,0.566647,46.292424,49.817704,-9.182378,49.592935,-53.707576,49.183275,10

In [13]:
data.isnull().sum()

timestamp         0
open              0
high              0
low               0
close             0
                 ..
Fisher            0
Fisher_trigger    0
supertrend        0
chop_index        0
Velocity          0
Length: 78, dtype: int64

In [ ]:
#data = data.drop_duplicates()

In [ ]:
#had to remove duplicates
#series = TimeSeries.from_dataframe(data,"timestamp", "close", freq='H')

# TRAIN SPLIT

In [14]:
train_size = 0.7
index = round(train_size*data.shape[0])

data_train = data.iloc[:index]
data_test = data.iloc[index:]

In [15]:
data_train.shape

(22789, 78)

In [16]:
data_test.shape

(9767, 78)

In [17]:
y_train=data_train[['timestamp', 'close']]
X_train=data_train.drop(columns=['timestamp', 'close'])
X_train

,open,high,low,volume,CMF,FII,MFI,VWA,AOI,KAMA,PPO,PPO_signal,PVO,PVO_signal,ROC,rsi,stoch_RSI,StochOscillator,StochOscillator_signal,TSI,UltimateOscillator,WilliamsRIndicator,ATR,BOL_lband,BOL_hband,BOL_avg,KCW,KCP,KCM,DCL,DCH,DCM,ULI,ADX,ADX_neg,ADX_pos,ARI_down,ARI,ARI_up,CCI,DPO,EMA_14,EMA_20,EMA_50,EMA_100,EMA_200,KST,KST_diff,KST_sig,MACD_line,MACD_signal,MAI,PSAR,SMA_14,SMA_20,SMA_50,SMA_100,SMA_200,STC,TRIX,vortex_ind_diff,vortex_neg,vortex_pos,wma_9,wma_20,wma_50,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,10216.35,10238.95,10194.39,872.764,-0.120924,-9.132667e+02,28.652105,10185.127432,-5.729912,10184.207475,-0.155468,-0.243303,1.037447,0.552894,0.598202,46.271721,0.676076,51.773363,58.093400,-10.639675,50.178602,-48.226637,52.300392,10073.766700,10358.878300,10216.3225,1.029615,0.418821,10217.990500,10080.7,10329.38,10205.040,1.452368,36.183688,30.858004,21.418009,60.0,-52.0,8.0,-3.877403,-79.6825,10216.809022,10223.590398,10253.805665,10258.150614,10223.457350,-10.009420,0.867360,-10.876780,-15.906877,-24.904204,26.531786,10100.005486,10181.362143,10216.3225,10269.6334,10292.0137,10247.86650,9.704022e+01,-0.038235,-0.167639,1.075610,0.907971,10203.142000,10203.709333,10246.597545,10275.790895,3005.70,2.25,1.7,1,0.45992,0.46011,-1,48.386232,-1
1,10208.81,10232.07,10204.53,819.923,-0.114688,9.179545e+02,36.501645,10187.270366,-12.684824,10186.551361,-0.137502,-0.222143,0.532956,0.548907,0.861528,48.644318,0.743204,57.612192,54.702161,-9.336736,54.098656,-42.387808,50.531792,10075.379977,10349.352023,10212.3660,1.031969,0.594886,10213.968500,10080.7,10329.38,10222.540,1.372308,34.888995,29.656738,20.584231,56.0,-52.0,4.0,6.904724,-92.2960,10217.763819,10223.626551,10252.635639,10257.473770,10223.462451,-9.424722,1.525039,-10.949762,-14.067871,-22.736938,26.587424,10104.592976,10186.853571,10212.3660,10267.6942,10290.3989,10248.98635,9.852011e+01,-0.035367,0.026085,1.064804,1.090889,10208.823048,10204.437667,10244.806824,10274.443495,3005.70,2.25,1.7,-1,0.40934,0.45992,-1,42.873022,-1
2,10224.11,10230.44,10187.88,922.804,-0.112046,8.026377e+02,36.306243,10191.939033,-26.571000,10188.754395,-0.121765,-0.202067,1.083131,0.655751,1.027859,48.664494,0.743775,57.660447,55.682001,-8.306903,51.810427,-42.339553,49.962379,10077.635517,10339.275483,10208.4555,1.049208,0.632549,10209.891000,10080.7,10329.38,10222.540,1.288889,33.967875,30.232613,19.331768,52.0,-44.0,8.0,5.018498,-64.1855,10218.607310,10223.670689,10251.516202,10256.812705,10223.468695,-8.210570,2.548815,-10.759384,-12.457165,-20.680983,26.638149,10109.088717,10192.235714,10208.4555,10265.5230,10288.4885,10248.40620,9.926006e+01,-0.032511,0.148340,0.933562,1.081902,10213.787905,10205.554238,10243.096855,10273.130448,3005.70,2.25,1.7,-1,0.31251,0.40934,-1,42.291553,-1
3,10225.02,10230.79,10192.69,905.598,-0.124361,-3.224208e+03,35.631136,10195.919801,-33.398088,10188.963245,-0.131656,-0.187985,1.339276,0.792456,0.488749,43.975832,0.611118,45.500241,53.590960,-8.853256,45.944458,-54.499759,49.115066,10078.361917,10328.766083,10203.5640,1.059908,0.394859,10205.222667,10080.7,10329.38,10224.335,1.199061,33.103085,28.557447,18.311510,48.0,-44.0,4.0,0.707552,-58.8340,10215.306335,10220.830623,10249.254783,10255.565919,10223.173982,-7.215594,3.074639,-10.290233,-13.465561,-19.237899,26.537221,10113.494542,10196.322143,10203.5640,10262.9314,10286.5536,10247.61835,9.963003e+01,-0.030300,0.189927,0.914300,1.104227,10214.003143,10204.163238,10240.286149,10271.256418,3005.70,2.25,1.7,-1,0.14629,0.31251,-1,41.595938,-1
4,10193.21,10215.07,10165.00,1089.829,-0.114965,-2.456898e+03,34.774165,10199.535893,-36.033412,10189.290069,-0.136376,-0.177663,3.169482,1.267861,0.503611,44.351977,0.566647,46.292424,49.817704,-9.182378,49.592935,-53.707576,49.183275,10081.522744,10314.952256,10198.2375,1.077188,0.463785,10199.799000,10080.7,10329.38,10224.335,1.079309,32.772767,30.502255,16.979964,44.0,

In [18]:
y_train

,timestamp,close
0,2019-09-17 04:00:00,10209.45
1,2019-09-17 05:00:00,10223.97
2,2019-09-17 06:00:00,10224.09
3,2019-09-17 07:00:00,10193.85
4,2019-09-17 08:00:00,10195.82
...,...,...
22784,2022-04-23 01:00:00,39683.50
22785,2022-04-23 02:00:00,39674.20
22786,2022-04-23 03:00:00,39544.90
22787,2022-04-23 04:00:00,39588.10


In [19]:
y_test=data_test[['timestamp', 'close']]
X_test=data_test.drop(columns=['timestamp', 'close'])
X_test

,open,high,low,volume,CMF,FII,MFI,VWA,AOI,KAMA,PPO,PPO_signal,PVO,PVO_signal,ROC,rsi,stoch_RSI,StochOscillator,StochOscillator_signal,TSI,UltimateOscillator,WilliamsRIndicator,ATR,BOL_lband,BOL_hband,BOL_avg,KCW,KCP,KCM,DCL,DCH,DCM,ULI,ADX,ADX_neg,ADX_pos,ARI_down,ARI,ARI_up,CCI,DPO,EMA_14,EMA_20,EMA_50,EMA_100,EMA_200,KST,KST_diff,KST_sig,MACD_line,MACD_signal,MAI,PSAR,SMA_14,SMA_20,SMA_50,SMA_100,SMA_200,STC,TRIX,vortex_ind_diff,vortex_neg,vortex_pos,wma_9,wma_20,wma_50,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
22789,39478.6,39534.9,39371.3,4266.286,-0.238306,-764779.952867,33.530461,39593.522591,-577.050000,39688.484217,-0.838194,-0.916076,-24.671183,-12.739861,0.061145,31.932106,0.689449,23.432470,25.623156,-35.516339,39.782391,-76.567530,310.616580,38975.138673,40605.391327,39790.265,1.682875,-0.060130,39813.998333,39133.0,40760.0,39510.65,2.539186,46.795621,28.857892,9.306124,60.0,-40.0,20.0,-65.925237,-431.165,39673.574575,39819.465320,40370.935891,40625.318645,40815.113079,-34.351552,0.212456,-34.564008,-334.979771,-367.800596,23.379905,39930.086978,39536.928571,39790.265,40724.402,40987.484,40550.3145,15.623160,-0.137624,-0.375702,1.241589,0.865887,39539.232381,39621.676190,40222.230510,40766.011010,4271.78,0.50,8.30,-1,-0.20516,0.18007,-1,47.242026,-1
22790,39439.8,39539.7,39259.0,9550.377,-0.177117,-556338.187043,37.802437,39505.473177,-577.784412,39687.571710,-0.800346,-0.892930,-22.595348,-14.710958,0.386950,35.042365,0.925292,43.449305,29.982126,-34.823446,47.063655,-56.550695,308.479681,38971.087310,40521.962690,39746.525,1.647030,0.118457,39761.265000,39133.0,40631.4,39552.00,2.481452,47.349033,29.582550,8.701261,56.0,-40.0,16.0,-64.838889,-431.725,39651.951298,39790.125765,40337.228601,40603.260850,40802.140809,-33.560358,1.142929,-34.703287,-319.585301,-358.157537,23.387123,39882.261759,39512.835714,39746.525,40682.768,40973.998,40548.4335,57.811580,-0.134283,-0.340934,1.215748,0.874814,39535.828571,39595.117619,40174.661804,40736.781624,4271.78,0.50,8.30,-1,-0.68198,-0.20516,-1,59.270247,-1
22791,39511.3,39598.0,39480.2,3847.574,-0.170829,-461141.215123,51.982243,39484.695592,-554.181765,39685.584027,-0.755855,-0.865515,-25.684116,-16.905590,0.572812,36.275963,1.000000,57.162921,41.348232,-33.913620,43.726214,-42.837079,294.859704,38970.063683,40448.576317,39709.320,1.608368,0.213772,39722.868333,39133.0,40631.4,39552.00,2.338434,47.459066,28.738365,9.865251,52.0,-40.0,12.0,-41.002290,-205.920,39637.024458,39766.304264,40305.964734,40582.206180,40789.582194,-32.086291,2.495486,-34.581777,-301.600628,-346.846155,23.385588,39837.306054,39521.792857,39709.320,40640.090,40962.088,40546.4225,78.905790,-0.130303,-0.063147,1.113513,1.050366,39539.450476,39575.448571,40129.847373,40708.385624,4271.78,0.50,8.30,1,-0.74952,-0.68198,-1,61.153135,-1
22792,39540.0,39656.7,39520.0,3824.316,-0.218877,-383189.986734,64.566231,39519.019505,-520.173824,39684.179523,-0.707956,-0.834003,-28.271598,-19.178792,0.148595,37.267361,1.000000,57.694033,52.768753,-32.884205,43.720782,-42.305967,283.562582,39005.394678,40325.165322,39665.280,1.550125,0.300973,39684.533333,39133.0,40631.4,39552.00,2.171177,47.155458,27.748779,11.004183,48.0,-40.0,8.0,-27.655883,-182.680,39627.034530,39746.856239,40276.793568,40562.006058,40777.368441,-30.406788,3.714109,-34.120897,-282.310065,-333.938937,23.413631,39795.047690,39536.292857,39665.280,40599.618,40951.820,40545.1185,89.452895,-0.125704,0.157480,0.958893,1.116373,39544.824762,39561.427619,40087.573255,40680.663089,4271.78,0.50,8.30,1,-0.57086,-0.74952,-1,60.780786,-1
22793,39562.0,39629.2,39525.3,2659.351,-0.251515,-322332.052757,58.859295,39537.513920,-472.450882,39683.657033,-0.659132,-0.799029,-31.596142,-21.662262,0.242132,38.023825,1.000000,60.101690,58.319548,-31.812733,51.033439,-39.898310,270.729541,39098.626650,40133.953350,39616.290,1.537445,0.402116,39637.851667,39133.0,40631.4,39552.00,1.976

In [20]:
y_test

,timestamp,close
22789,2022-04-23 06:00:00,39438.7
22790,2022-04-23 07:00:00,39511.4
22791,2022-04-23 08:00:00,39540.0
22792,2022-04-23 09:00:00,39562.1
22793,2022-04-23 10:00:00,39578.2
...,...,...
32551,2023-06-03 20:00:00,27147.6
32552,2023-06-03 21:00:00,27138.8
32553,2023-06-03 22:00:00,27084.9
32554,2023-06-03 23:00:00,27052.7


# ETS

In [ ]:
from darts.models import ExponentialSmoothing

In [ ]:
model = ExponentialSmoothing()
model.fit(train)
prediction = model.predict(len(val), num_samples=1000)

In [ ]:
import matplotlib.pyplot as plt

series.plot()
prediction.plot(label="forecast", low_quantile=0.05, high_quantile=0.95)
plt.legend()

In [ ]:
from darts.datasets import ETTh2Dataset

series = ETTh2Dataset().load()[:10000][["MUFL", "LULL"]]
train, val = series.split_before(0.6)

In [ ]:
from darts.ad import KMeansScorer

scorer = KMeansScorer(k=2, window=5)
scorer.fit(train)
anom_score = scorer.score(val)

In [ ]:
from darts.ad import QuantileDetector

detector = QuantileDetector(high_quantile=0.99)
detector.fit(scorer.score(train))
binary_anom = detector.detect(anom_score)

In [ ]:
import matplotlib.pyplot as plt

series.plot()
(anom_score / 2. - 100).plot(label="computed anomaly score", c="orangered", lw=3)
(binary_anom * 45 - 150).plot(label="detected binary anomaly", lw=4)

In [ ]:
from darts.models import Theta

In [ ]:
model = Theta(theta = 1)

In [ ]:
pred = model.predict(len(val))

In [ ]:
from darts.metrics import mape

In [ ]:
score=mape(val,pred)

In [ ]:
from darts.metrics import mase

In [ ]:
score=mase(val,pred,train)

In [ ]:
historical_forecast = model.historical_forecasts(series = series,start = 0.5,forecast_horizon=12)

Choosing the model for forecasting

Gridsearch best model.best params

In [ ]:
parametersm={'theta':[0.5,1,1.5,2,2.5],
             'season_mode':[SeasonalityMode.MULTIPLICATIVE,
                            SeasonalityMode.ADDITIVE]}

# ARIMA

In [21]:
from statsmodels.tsa.arima.model import ARIMA

In [22]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [23]:
data.set_index('timestamp',inplace=True)

In [24]:
data

,open,high,low,close,volume,CMF,FII,MFI,VWA,AOI,KAMA,PPO,PPO_signal,PVO,PVO_signal,ROC,rsi,stoch_RSI,StochOscillator,StochOscillator_signal,TSI,UltimateOscillator,WilliamsRIndicator,ATR,BOL_lband,BOL_hband,BOL_avg,KCW,KCP,KCM,DCL,DCH,DCM,ULI,ADX,ADX_neg,ADX_pos,ARI_down,ARI,ARI_up,CCI,DPO,EMA_14,EMA_20,EMA_50,EMA_100,EMA_200,KST,KST_diff,KST_sig,MACD_line,MACD_signal,MAI,PSAR,SMA_14,SMA_20,SMA_50,SMA_100,SMA_200,STC,TRIX,vortex_ind_diff,vortex_neg,vortex_pos,wma_9,wma_20,wma_50,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-09-17 04:00:00,10216.35,10238.95,10194.39,10209.45,872.764,-0.120924,-913.266701,28.652105,10185.127432,-5.729912,10184.207475,-0.155468,-0.243303,1.037447,0.552894,0.598202,46.271721,0.676076,51.773363,58.093400,-10.639675,50.178602,-48.226637,52.300392,10073.766700,10358.878300,10216.3225,1.029615,0.418821,10217.990500,10080.7,10329.38,10205.040,1.452368,36.183688,30.858004,21.418009,60.0,-52.0,8.0,-3.877403,-79.6825,10216.809022,10223.590398,10253.805665,10258.150614,10223.457350,-10.009420,0.867360,-10.876780,-15.906877,-24.904204,26.531786,10100.005486,10181.362143,10216.3225,10269.6334,10292.0137,10247.86650,97.040222,-0.038235,-0.167639,1.075610,0.907971,10203.142000,10203.709333,10246.597545,10275.790895,3005.70,2.25,1.70,1,0.45992,0.46011,-1,48.386232,-1
2019-09-17 05:00:00,10208.81,10232.07,10204.53,10223.97,819.923,-0.114688,917.954536,36.501645,10187.270366,-12.684824,10186.551361,-0.137502,-0.222143,0.532956,0.548907,0.861528,48.644318,0.743204,57.612192,54.702161,-9.336736,54.098656,-42.387808,50.531792,10075.379977,10349.352023,10212.3660,1.031969,0.594886,10213.968500,10080.7,10329.38,10222.540,1.372308,34.888995,29.656738,20.584231,56.0,-52.0,4.0,6.904724,-92.2960,10217.763819,10223.626551,10252.635639,10257.473770,10223.462451,-9.424722,1.525039,-10.949762,-14.067871,-22.736938,26.587424,10104.592976,10186.853571,10212.3660,10267.6942,10290.3989,10248.98635,98.520111,-0.035367,0.026085,1.064804,1.090889,10208.823048,10204.437667,10244.806824,10274.443495,3005.70,2.25,1.70,-1,0.40934,0.45992,-1,42.873022,-1
2019-09-17 06:00:00,10224.11,10230.44,10187.88,10224.09,922.804,-0.112046,802.637671,36.306243,10191.939033,-26.571000,10188.754395,-0.121765,-0.202067,1.083131,0.655751,1.027859,48.664494,0.743775,57.660447,55.682001,-8.306903,51.810427,-42.339553,49.962379,10077.635517,10339.275483,10208.4555,1.049208,0.632549,10209.891000,10080.7,10329.38,10222.540,1.288889,33.967875,30.232613,19.331768,52.0,-44.0,8.0,5.018498,-64.1855,10218.607310,10223.670689,10251.516202,10256.812705,10223.468695,-8.210570,2.548815,-10.759384,-12.457165,-20.680983,26.638149,10109.088717,10192.235714,10208.4555,10265.5230,10288.4885,10248.40620,99.260055,-0.032511,0.148340,0.933562,1.081902,10213.787905,10205.554238,10243.096855,10273.130448,3005.70,2.25,1.70,-1,0.31251,0.40934,-1,42.291553,-1
2019-09-17 07:00:00,10225.02,10230.79,10192.69,10193.85,905.598,-0.124361,-3224.208213,35.631136,10195.919801,-33.398088,10188.963245,-0.131656,-0.187985,1.339276,0.792456,0.488749,43.975832,0.611118,45.500241,53.590960,-8.853256,45.944458,-54.499759,49.115066,10078.361917,10328.766083,10203.5640,1.059908,0.394859,10205.222667,10080.7,10329.38,10224.335,1.199061,33.103085,28.557447,18.311510,48.0,-44.0,4.0,0.707552,-58.8340,10215.306335,10220.830623,10249.254783,10255.565919,10223.173982,-7.215594,3.074639,-10.290233,-13.465561,-19.237899,26.537221,10113.494542,10196.322143,10203.5640,10262.9314,10286.5536,10247.61835,99.630028,-0.030300,0.189927,0.914300,1.104227,10214.003143,10204.163238,10240.286149,10271.256418,3005.70,2.25,1.70,-1,0.14629,0.31251,-1,41.595938,-1
2019-09-17 08:00:00,10193.21,10215.07,10165.00,10195.82,1089.829,-0.114965,-2456.898021,34.774165,10199.535893,-36.033412,10189.290069,-0.136376,-0.177663,3.169482,1.267861,0.503611,44.3519

In [25]:
type(data.index)

pandas.core.indexes.base.Index

Multiplicative Decomposition 

In [26]:
indexed_data=data[:1000]

In [27]:
indexed_data

,open,high,low,close,volume,CMF,FII,MFI,VWA,AOI,KAMA,PPO,PPO_signal,PVO,PVO_signal,ROC,rsi,stoch_RSI,StochOscillator,StochOscillator_signal,TSI,UltimateOscillator,WilliamsRIndicator,ATR,BOL_lband,BOL_hband,BOL_avg,KCW,KCP,KCM,DCL,DCH,DCM,ULI,ADX,ADX_neg,ADX_pos,ARI_down,ARI,ARI_up,CCI,DPO,EMA_14,EMA_20,EMA_50,EMA_100,EMA_200,KST,KST_diff,KST_sig,MACD_line,MACD_signal,MAI,PSAR,SMA_14,SMA_20,SMA_50,SMA_100,SMA_200,STC,TRIX,vortex_ind_diff,vortex_neg,vortex_pos,wma_9,wma_20,wma_50,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-09-17 04:00:00,10216.35,10238.95,10194.39,10209.45,872.764,-0.120924,-913.266701,28.652105,10185.127432,-5.729912,10184.207475,-0.155468,-0.243303,1.037447,0.552894,0.598202,46.271721,0.676076,51.773363,58.093400,-10.639675,50.178602,-48.226637,52.300392,10073.766700,10358.878300,10216.3225,1.029615,0.418821,10217.990500,10080.70,10329.38,10205.040,1.452368,36.183688,30.858004,21.418009,60.0,-52.0,8.0,-3.877403,-79.6825,10216.809022,10223.590398,10253.805665,10258.150614,10223.457350,-10.009420,0.867360,-10.876780,-15.906877,-24.904204,26.531786,10100.005486,10181.362143,10216.3225,10269.6334,10292.0137,10247.86650,97.040222,-0.038235,-0.167639,1.075610,0.907971,10203.142000,10203.709333,10246.597545,10275.790895,3005.70,2.25,1.7,1,0.45992,0.46011,-1,48.386232,-1
2019-09-17 05:00:00,10208.81,10232.07,10204.53,10223.97,819.923,-0.114688,917.954536,36.501645,10187.270366,-12.684824,10186.551361,-0.137502,-0.222143,0.532956,0.548907,0.861528,48.644318,0.743204,57.612192,54.702161,-9.336736,54.098656,-42.387808,50.531792,10075.379977,10349.352023,10212.3660,1.031969,0.594886,10213.968500,10080.70,10329.38,10222.540,1.372308,34.888995,29.656738,20.584231,56.0,-52.0,4.0,6.904724,-92.2960,10217.763819,10223.626551,10252.635639,10257.473770,10223.462451,-9.424722,1.525039,-10.949762,-14.067871,-22.736938,26.587424,10104.592976,10186.853571,10212.3660,10267.6942,10290.3989,10248.98635,98.520111,-0.035367,0.026085,1.064804,1.090889,10208.823048,10204.437667,10244.806824,10274.443495,3005.70,2.25,1.7,-1,0.40934,0.45992,-1,42.873022,-1
2019-09-17 06:00:00,10224.11,10230.44,10187.88,10224.09,922.804,-0.112046,802.637671,36.306243,10191.939033,-26.571000,10188.754395,-0.121765,-0.202067,1.083131,0.655751,1.027859,48.664494,0.743775,57.660447,55.682001,-8.306903,51.810427,-42.339553,49.962379,10077.635517,10339.275483,10208.4555,1.049208,0.632549,10209.891000,10080.70,10329.38,10222.540,1.288889,33.967875,30.232613,19.331768,52.0,-44.0,8.0,5.018498,-64.1855,10218.607310,10223.670689,10251.516202,10256.812705,10223.468695,-8.210570,2.548815,-10.759384,-12.457165,-20.680983,26.638149,10109.088717,10192.235714,10208.4555,10265.5230,10288.4885,10248.40620,99.260055,-0.032511,0.148340,0.933562,1.081902,10213.787905,10205.554238,10243.096855,10273.130448,3005.70,2.25,1.7,-1,0.31251,0.40934,-1,42.291553,-1
2019-09-17 07:00:00,10225.02,10230.79,10192.69,10193.85,905.598,-0.124361,-3224.208213,35.631136,10195.919801,-33.398088,10188.963245,-0.131656,-0.187985,1.339276,0.792456,0.488749,43.975832,0.611118,45.500241,53.590960,-8.853256,45.944458,-54.499759,49.115066,10078.361917,10328.766083,10203.5640,1.059908,0.394859,10205.222667,10080.70,10329.38,10224.335,1.199061,33.103085,28.557447,18.311510,48.0,-44.0,4.0,0.707552,-58.8340,10215.306335,10220.830623,10249.254783,10255.565919,10223.173982,-7.215594,3.074639,-10.290233,-13.465561,-19.237899,26.537221,10113.494542,10196.322143,10203.5640,10262.9314,10286.5536,10247.61835,99.630028,-0.030300,0.189927,0.914300,1.104227,10214.003143,10204.163238,10240.286149,10271.256418,3005.70,2.25,1.7,-1,0.14629,0.31251,-1,41.595938,-1
2019-09-17 08:00:00,10193.21,10215.07,10165.00,10195.82,1089.829,-0.114965,-2456.898021,34.774165,10199.535893,-36.033412,10189.290069,-0.136376,-0.177663,3.169482,1.267861,0.503611,44.3519

In [36]:
result_mul = seasonal_decompose(data['close'], model='multiplicative',period=120)

In [29]:
pwd!

'/Users/IRIS/code/MehmetSever09/assist_trader/notebook'

In [37]:
#Removing seasons
data['deseasonalized'] = data.close.values/result_mul.seasonal

In [ ]:
#plt.figure(figsize=(15,4)); plt.subplot(1,2,1); plt.plot(data.deseasonalized);
#plt.title('close price lin', fontsize=16);

In [38]:
#Removing trend
data['linearized'] = np.log(data['deseasonalized'])

In [39]:
#plt.subplot(1,2,2); plt.plot(data['linearized'])
#plt.title('close price lin', fontsize=16);

In [43]:
# 1st Differencing
y_diff = data['linearized'].diff().dropna()

# 2nd Differencing
y_diff_diff = data['linearized'].diff().diff().dropna()

In [45]:
print('p-value zero-diff: ', adfuller(data['linearized'])[1])
print('p-value first-diff: ', adfuller(data['linearized'].diff().dropna())[1])
print('p-value second-diff: ', adfuller(data['linearized'].diff().diff().dropna())[1])

p-value zero-diff:  0.6267091734154964
p-value first-diff:  0.0
p-value second-diff:  0.0


In [46]:
from pmdarima.arima.utils import ndiffs
ndiffs(data['linearized'])

1

In [52]:
arima = ARIMA(data['linearized'], order=(2, 1, 0), trend='t')
arima = arima.fit()

In [53]:
arima.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:             linearized   No. Observations:                32556
Model:                 ARIMA(2, 1, 0)   Log Likelihood              112300.076
Date:                Thu, 08 Jun 2023   AIC                        -224592.152
Time:                        17:36:00   BIC                        -224558.589
Sample:                             0   HQIC                       -224581.424
                              - 32556                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1          2.979e-05   4.19e-05      0.711      0.477   -5.23e-05       0.000
ar.L1         -0.0148      0.002     -8.401      0.000      -0.018      -0.011
ar.L2         -0.0195      0.002    -11.063      0.000      -0.023      -0.016
sigma2      5.906e-05   8.95e-08    660.205      0.000    5.89e-05    5.92e-05
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):           3859767.23
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.64   Skew:                            -1.08
Prob(H) (two-sided):                  0.00   Kurtosis:                        56.30
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

parameters

In [51]:
import pmdarima as pm
smodel = pm.auto_arima(data['linearized'],
                       start_p=1, max_p=2,
                       start_q=1, max_q=2,
                       trend='t',
                       seasonal=False,
                       trace=True)

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-224585.391, Time=14.65 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-224576.507, Time=6.29 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-224581.393, Time=8.31 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-224581.665, Time=11.34 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-224576.507, Time=6.32 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-224590.562, Time=17.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-224591.705, Time=14.62 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=-224591.705, Time=13.58 sec

Best model:  ARIMA(2,1,0)(0,0,0)[0]          
Total fit time: 92.173 seconds


In [54]:
data['linearized']

timestamp
2019-09-17 04:00:00     9.229966
2019-09-17 05:00:00     9.231218
2019-09-17 06:00:00     9.231617
2019-09-17 07:00:00     9.229126
2019-09-17 08:00:00     9.229587
                         ...    
2023-06-03 20:00:00    10.208953
2023-06-03 21:00:00    10.208893
2023-06-03 22:00:00    10.206971
2023-06-03 23:00:00    10.206184
2023-06-04 00:00:00    10.203914
Name: linearized, Length: 32556, dtype: float64

In [55]:
train = data['linearized'][0:150]
test = data['linearized'][150:]

# Building model
arima = ARIMA(train, order=(2, 1, 0), trend='t')  
arima = arima.fit()


forecast = arima.forecast(len(test), alpha=0.05)  # 95% confidence


forecast_results = arima.get_forecast(len(test), alpha=0.05)
forecast = forecast_results.predicted_mean
confidence_int = forecast_results.conf_int().values

In [59]:
def plot_forecast(fc, train, test, upper=None, lower=None):
    is_confidence_int = isinstance(upper, np.ndarray) and isinstance(lower, np.ndarray)
    # Prepare plot
    fc_series = pd.Series(fc, index=test.index)
    lower_series = pd.Series(upper, index=test.index) if is_confidence_int else None
    upper_series = pd.Series(lower, index=test.index) if is_confidence_int else None

In [ ]:
plt.figure(figsize=(10,4), dpi=100)
plt.plot(train, label='training', color='black')
plt.plot(test, label='actual', color='black', ls='--')
plt.plot(fc_series, label='forecast', color='orange')
if is_confidence_int:
    plt.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.15)
    plt.title('Forecast vs Actuals')
    plt.legend(loc='upper left', fontsize=8);

NameError: name 'fc_series' is not defined

In [ ]:
plot_forecast(forecast, train, test, confidence_int[:,0], confidence_int[:,1])

# SARIMA

Train split

In [ ]:
data_test['log'] = np.log(data_test.close)

In [ ]:
train = data_test.log[0:150]
test = data_test.log[150:]

In [ ]:
smodel = pm.auto_arima(train, seasonal=True, m=12, 
                       start_p=0, max_p=1, max_d=1, start_q=0, max_q=1,
                       start_P=0, max_P=2, max_D=1, start_Q=0, max_Q=2, 
                       trace=True, error_action='ignore', suppress_warnings=True)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX


sarima = SARIMAX(train, order=(0, 1, 0), seasonal_order=(1, 0, 0, 12))
sarima = sarima.fit(maxiter=75)

results = sarima.get_forecast(len(test), alpha=0.05)
forecast = results.predicted_mean
confidence_int = results.conf_int()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

sarima = SARIMAX(train, order=(0, 1, 0), seasonal_order=(1, 0, 0, 12))
sarima = sarima.fit(maxiter=75)

results = sarima.get_forecast(len(test), alpha=0.05)
forecast = results.predicted_mean
confidence_int = results.conf_int()

In [ ]:
sarima.summary()

# SARIMAX